Importing necessary libraries that can be used.

In [ ]:
import pandas as pd

Importing the dataset to the google colab. It will also show the 6 columns of the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/ELECTIVE 4/Exercise 2/universal_studio_branches.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,reviewer,rating,written_date,title,review_text,branch
0,Kelly B,2.0,"May 30, 2021",Universal is a complete Disaster - stick with ...,We went to Universal over Memorial Day weekend...,Universal Studios Florida
1,Jon,1.0,"May 30, 2021",Food is hard to get.,The food service is horrible. I’m not reviewin...,Universal Studios Florida
2,Nerdy P,2.0,"May 30, 2021",Disappointed,I booked this vacation mainly to ride Hagrid m...,Universal Studios Florida
3,ran101278,4.0,"May 29, 2021",My opinion,When a person tries the test seat for the ride...,Universal Studios Florida
4,tammies20132015,5.0,"May 28, 2021",The Bourne Stuntacular...MUST SEE,"Ok, I can't stress enough to anyone and everyo...",Universal Studios Florida


#Data Cleaning

##Checking Null Values

Checking if there is a null values in the dataset

In [ ]:
df.isnull().sum()

,0
reviewer,0
rating,0
written_date,0
title,0
review_text,0
branch,0


##Checking Unique Values

Checking the UNIQUE VALUES that are present in the dataset. Since there are only 44086 unique values and review text has 50847 unique values in the review_text column, we can say that there are some reviewer did review for more than 1 time, which can be valid since they can visit the universal studio anytime.

In [ ]:
for col in df.columns:
  print(f"{col}: {df[col].unique().size} unique values")

reviewer: 44086 unique values
rating: 5 unique values
written_date: 4276 unique values
title: 37522 unique values
review_text: 50847 unique values
branch: 3 unique values


##Removing Duplicate Data

In [ ]:
num_duplicates = df.duplicated(subset=['review_text']).sum()
print("Duplicate Reviews:", num_duplicates)

# Remove rows with duplicate 'review_text' values
df_no_duplicates = df.drop_duplicates(subset=['review_text'])

# Verify the number of rows after removing duplicates
print("Update Rows:", df_no_duplicates.shape[0])

#Apply changes to the dataset
df = df_no_duplicates


Duplicate Reviews: 57
Update Rows: 50847


In [ ]:
df.shape

(50847, 6)

In [ ]:
df['review_text']

,review_text
0,We went to Universal over Memorial Day weekend...
1,The food service is horrible. I’m not reviewin...
2,I booked this vacation mainly to ride Hagrid m...
3,When a person tries the test seat for the ride...
4,"Ok, I can't stress enough to anyone and everyo..."
...,...
50899,This is my first visit to a Universal Studio t...
50900,We finally visited Singapore's very first them...
50901,We visited during the first week of its 'soft ...
50902,We visited on the 3rd day of the 'soft' openin...


##Normalizing Reviewer Name

Showing the initial inputs from the reviewer here, it is noticeable that it has inconsistent data format which makes it hard to understand.

In [ ]:
df['reviewer']

,reviewer
0,Kelly B
1,Jon
2,Nerdy P
3,ran101278
4,tammies20132015
...,...
50899,vinz20
50900,betty l
50901,spoonos65
50902,HeatSeekerWrexham_UK


Making the reviewer name lowercase to ensure that all names have the same format and replacing all underscore symbol which are usually used in names into a white space

In [ ]:
df['reviewer'] = df['reviewer'].str.lower().str.replace('_', ' ')

<ipython-input-620-296c8510d2f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewer'] = df['reviewer'].str.lower().str.replace('_', ' ')


In [ ]:
df

,reviewer,rating,written_date,title,review_text,branch
0,kelly b,2.0,"May 30, 2021",Universal is a complete Disaster - stick with ...,We went to Universal over Memorial Day weekend...,Universal Studios Florida
1,jon,1.0,"May 30, 2021",Food is hard to get.,The food service is horrible. I’m not reviewin...,Universal Studios Florida
2,nerdy p,2.0,"May 30, 2021",Disappointed,I booked this vacation mainly to ride Hagrid m...,Universal Studios Florida
3,ran101278,4.0,"May 29, 2021",My opinion,When a person tries the test seat for the ride...,Universal Studios Florida
4,tammies20132015,5.0,"May 28, 2021",The Bourne Stuntacular...MUST SEE,"Ok, I can't stress enough to anyone and everyo...",Universal Studios Florida
...,...,...,...,...,...,...
50899,vinz20,4.0,"March 29, 2010",I'll Be Back Only If ...,This is my first visit to a Universal Studio t...,Universal Studios Singapore
50900,betty l,4.0,"March 29, 2010",Universal Studios Singapore Experience,We finally visited Singapore's very first them...,Universal Studios Singapore
50901,spoonos65,4.0,"March 28, 2010",Impressive but not quite finished!,We visited during the first week of its 'soft ...,Universal Studios Singapore
50902,heatseekerwrexham uk,4.0,"March 22, 2010",Small but beautifully marked,We visited on the 3rd day of the 'soft' openin...,Universal Studios Singapore


Separating the words and numbers if it is combined in order to easily break down the name of the reviewer

In [ ]:
import re

def remove_numbers(name):
  return re.sub(r'\d+', '', name).strip()

df['reviewer'] = df['reviewer'].apply(remove_numbers)
df['reviewer']

,reviewer
0,kelly b
1,jon
2,nerdy p
3,ran
4,tammies
...,...
50899,vinz
50900,betty l
50901,spoonos
50902,heatseekerwrexham uk


In [ ]:

df['reviewer']


,reviewer
0,kelly b
1,jon
2,nerdy p
3,ran
4,tammies
...,...
50899,vinz
50900,betty l
50901,spoonos
50902,heatseekerwrexham uk


##Standardizing the date

Changing the date into a certain format Month-Day-Year.

In [ ]:
df['written_date'] = pd.to_datetime(df['written_date']).dt.strftime('%m-%d-%Y')
df['written_date']

,written_date
0,05-30-2021
1,05-30-2021
2,05-30-2021
3,05-29-2021
4,05-28-2021
...,...
50899,03-29-2010
50900,03-29-2010
50901,03-28-2010
50902,03-22-2010


In [ ]:
df.shape

(50847, 6)

#Text Pre-Processing Techniques

In [ ]:
df1 = df.copy()
df1['review_text']

,review_text
0,We went to Universal over Memorial Day weekend...
1,The food service is horrible. I’m not reviewin...
2,I booked this vacation mainly to ride Hagrid m...
3,When a person tries the test seat for the ride...
4,"Ok, I can't stress enough to anyone and everyo..."
...,...
50899,This is my first visit to a Universal Studio t...
50900,We finally visited Singapore's very first them...
50901,We visited during the first week of its 'soft ...
50902,We visited on the 3rd day of the 'soft' openin...


## Lowercasing

Creating a copy of df to ensure to have a backup of originial file. And converting each row in 'review_text' column in lowercase.

In [ ]:

df1['review_text'] = df1['review_text'].str.lower()
df1['review_text']

,review_text
0,we went to universal over memorial day weekend...
1,the food service is horrible. i’m not reviewin...
2,i booked this vacation mainly to ride hagrid m...
3,when a person tries the test seat for the ride...
4,"ok, i can't stress enough to anyone and everyo..."
...,...
50899,this is my first visit to a universal studio t...
50900,we finally visited singapore's very first them...
50901,we visited during the first week of its 'soft ...
50902,we visited on the 3rd day of the 'soft' openin...


This code will show the lowercase format of the first row.

In [ ]:
df1['review_text'].iloc[0]

'we went to universal over memorial day weekend and it was a total train wreck. we waited to get in the parking lot for about forty minutes. we paid for prime parking to make up for all the wasted time.  then we paid extra for the express pass 2-park tickets only to be turned away and sent to guest services bc the app didn’t show the bar code. the line at guest services took forever. they are clearly understaffed. we were sent to yet another guest services line because we had the express passes! also took ages! we spent nearly 2 hours just trying to enter the park! when we shared this with jackie at guest services she smirked, didn’t even apologize and was patronizing. this would never happen at disney.  once inside several of the rides didn’t work, when they reopened they were backed up so the express line on some rides was still a full hour wait and two hours without the express pass! we also saw people jump over and sneak in to the express lanes and then convince the workers to just

## Removing Special Characters

In [ ]:
df1['review_text'].iloc[0]

'we went to universal over memorial day weekend and it was a total train wreck. we waited to get in the parking lot for about forty minutes. we paid for prime parking to make up for all the wasted time.  then we paid extra for the express pass 2-park tickets only to be turned away and sent to guest services bc the app didn’t show the bar code. the line at guest services took forever. they are clearly understaffed. we were sent to yet another guest services line because we had the express passes! also took ages! we spent nearly 2 hours just trying to enter the park! when we shared this with jackie at guest services she smirked, didn’t even apologize and was patronizing. this would never happen at disney.  once inside several of the rides didn’t work, when they reopened they were backed up so the express line on some rides was still a full hour wait and two hours without the express pass! we also saw people jump over and sneak in to the express lanes and then convince the workers to just

This shows the removal of sepcial characters testing in the first row in 'review_text' column. Anything that will not match the pattern '[^a-zA-Z0-9\s]' will be removed.

In [ ]:
import re

# Removing special characters from the first row of 'review_text'
text = df1['review_text'].iloc[0]
text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
print(text)


we went to universal over memorial day weekend and it was a total train wreck we waited to get in the parking lot for about forty minutes we paid for prime parking to make up for all the wasted time  then we paid extra for the express pass 2park tickets only to be turned away and sent to guest services bc the app didnt show the bar code the line at guest services took forever they are clearly understaffed we were sent to yet another guest services line because we had the express passes also took ages we spent nearly 2 hours just trying to enter the park when we shared this with jackie at guest services she smirked didnt even apologize and was patronizing this would never happen at disney  once inside several of the rides didnt work when they reopened they were backed up so the express line on some rides was still a full hour wait and two hours without the express pass we also saw people jump over and sneak in to the express lanes and then convince the workers to just let them on if the

This is the application of same process to the entire rows of the 'review_text' column. Using the function lambda, it takes each value of (x) which is the values from ['review_text'] column to apply the re.sub(r'[^a-zA-Z0-9\s]'.

In [ ]:
df1['review_text'] = df1['review_text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
df1['review_text'].iloc[0]

'we went to universal over memorial day weekend and it was a total train wreck we waited to get in the parking lot for about forty minutes we paid for prime parking to make up for all the wasted time  then we paid extra for the express pass 2park tickets only to be turned away and sent to guest services bc the app didnt show the bar code the line at guest services took forever they are clearly understaffed we were sent to yet another guest services line because we had the express passes also took ages we spent nearly 2 hours just trying to enter the park when we shared this with jackie at guest services she smirked didnt even apologize and was patronizing this would never happen at disney  once inside several of the rides didnt work when they reopened they were backed up so the express line on some rides was still a full hour wait and two hours without the express pass we also saw people jump over and sneak in to the express lanes and then convince the workers to just let them on if th

##Removing Stopwords

Importing necessary library for stopwords. It will download all of the existing stopwords in the package 'stopwords'.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creating a function that can be call in the next lines of code to remove the stopwords that will match the words that are stored in 'stop_words' variable. And if the word does not match in any word in stop_words variable it will be storedin fitered_words.

In [ ]:
def remove_stopwords(text):
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  return ' '.join(filtered_words)

Testing the function to the previous text variable which as the first row of 'review_text' column to see if the function will work.

In [ ]:
text = remove_stopwords(text)
print(text)

went universal memorial day weekend total train wreck waited get parking lot forty minutes paid prime parking make wasted time paid extra express pass 2park tickets turned away sent guest services bc app didnt show bar code line guest services took forever clearly understaffed sent yet another guest services line express passes also took ages spent nearly 2 hours trying enter park shared jackie guest services smirked didnt even apologize patronizing would never happen disney inside several rides didnt work reopened backed express line rides still full hour wait two hours without express pass also saw people jump sneak express lanes convince workers let check point worked felt like complete suckers paying express pass still left us long lines people didnt pay could sneak long lines buy water use restroom get butter beer sucked horrible day avoid place total disaster


Application of the function to all entries of the 'review_text' column.

In [ ]:
df1['review_text'] = df1['review_text'].apply(remove_stopwords)
df1['review_text'].iloc[0]

'went universal memorial day weekend total train wreck waited get parking lot forty minutes paid prime parking make wasted time paid extra express pass 2park tickets turned away sent guest services bc app didnt show bar code line guest services took forever clearly understaffed sent yet another guest services line express passes also took ages spent nearly 2 hours trying enter park shared jackie guest services smirked didnt even apologize patronizing would never happen disney inside several rides didnt work reopened backed express line rides still full hour wait two hours without express pass also saw people jump sneak express lanes convince workers let check point worked felt like complete suckers paying express pass still left us long lines people didnt pay could sneak long lines buy water use restroom get butter beer sucked horrible day avoid place total disaster'

##Tokenization

This imports the 'word_tokenize' package that will allow tokenization of the text that is required to be tokenized. Creating new column for only tokenized text.

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
df1['tokenized_text'] = df1['review_text'].apply(word_tokenize)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(df1['tokenized_text'])

0        [went, universal, memorial, day, weekend, tota...
1        [food, service, horrible, im, reviewing, food,...
2        [booked, vacation, mainly, ride, hagrid, motor...
3        [person, tries, test, seat, rides, gets, green...
4        [ok, cant, stress, enough, anyone, everyone, g...
                               ...                        
50899    [first, visit, universal, studio, theme, park,...
50900    [finally, visited, singapores, first, theme, p...
50901    [visited, first, week, soft, opening, unfortun...
50902    [visited, 3rd, day, soft, opening, ticket, sal...
50903    [group, managed, get, tickets, 16, february, 2...
Name: tokenized_text, Length: 50847, dtype: object
